<br><img src="Grenoble INP - Ense3.png" style="width:342px;height:131px"/>
<br>
 ---
 
 
<h1><center>Dynamic Models for Building Energy Management</center></h1>
<h3><center> Antonin ANDRÉ - Guillaume DAUENDORFFER - Emma HOAREAU - Kelian JAMBU</center></h3>

---


## 1) Model

### Plan of a simple building

Here is the building design we chose:

<img src="Indoorair1.jpg" style="width:351px;height:365px"/>
<i><center>Fig. 1 - Planview of the building</center></i>
<br>
We decided to model our building with two identical rooms separated by a common wall. Each room has 3 walls of 3m each and a window of 3m as well.


### Hypothesis for boundary conditions, windows, doors, and wall composition.


### Adjancy matrix A, the conductance matrix G and the capacity matrix C.


## 2) Pyhton implementation: steady-state

### Implementation of matrices A, G, and C.
### Solar flows.
### Input vectors b and f in time.
### Steady-state response.
